In [4]:
from pprint import pprint
import time
import json
from bs4 import BeautifulSoup # For HTML parsing
import requests # Website connections
from time import sleep # To prevent overwhelming the server between connections
from collections import Counter # Keep track of our term counts
import pandas as pd # For converting results to a dataframe and bar chart plots
import numpy as np
import json # For parsing json
import urllib
from urllib.request import urlopen
import re
import matplotlib.pylab as plt
from datetime import datetime
from tqdm import tqdm_notebook as tqdm
%matplotlib inline

## IMDB Top 10k Movie Retrieval

In [3]:
# Automate the process of Top 10k movies website retrieval

def get_page(year,page):
    page = (page-1)*50+1
    url = f'https://www.imdb.com/search/title?title_type=feature&year={year}-01-01,{year}-12-31&sort=boxoffice_gross_us,desc&start={page}&ref_=adv_nxt'
    result=requests.get(url)
    soup=BeautifulSoup(result.text,'lxml')
    
    return soup

In [4]:
# Retieve the top 10k movie name and hyperlink
Movie_List=[]     # list of top 10k movies name
Link_List=[]      # list of top 10k movies link

for year in tqdm(range(1993,2019)):

    for page in range(1,10):
        soup=get_page(year,page)
        Movies = soup.find('div',class_='lister-list')

        for movie in Movies.find_all('h3',class_='lister-item-header'):
            M=movie.a.get_text(strip=True)
            Movie_List.append(M) 

        for link in Movies.find_all('h3',class_='lister-item-header'):
            Links = link.a.get('href')
            Full_Link = 'https://www.imdb.com{}'.format(Links)
            Link_List.append(Full_Link)    

KeyboardInterrupt: 

In [ ]:
Movie_List

In [ ]:
Title=pd.DataFrame({'Title':Movie_List,'Link':Link_List})
Title.to_csv('Title.csv',index=False)

## Read In top 10k movie file

In [5]:
Title=pd.read_csv('Title.csv')
Title.head(20)

,Title,Link
0,Jurassic Park,https://www.imdb.com/title/tt0107290/?ref_=adv...
1,Mrs. Doubtfire,https://www.imdb.com/title/tt0107614/?ref_=adv...
2,The Fugitive,https://www.imdb.com/title/tt0106977/?ref_=adv...
3,The Firm,https://www.imdb.com/title/tt0106918/?ref_=adv...
4,Sleepless in Seattle,https://www.imdb.com/title/tt0108160/?ref_=adv...
5,Indecent Proposal,https://www.imdb.com/title/tt0107211/?ref_=adv...
6,In the Line of Fire,https://www.imdb.com/title/tt0107206/?ref_=adv...
7,The Pelican Brief,https://www.imdb.com/title/tt0107798/?ref_=adv...
8,Schindler's List,https://www.imdb.com/title/tt0108052/?ref_=adv...
9,Cliffhanger,https://www.imdb.com/title/tt0106582/?ref_=adv...


In [6]:
Link_List_R=list(Title['Link'])
Link_List_R

['https://www.imdb.com/title/tt0107290/?ref_=adv_li_tt',
 'https://www.imdb.com/title/tt0107614/?ref_=adv_li_tt',
 'https://www.imdb.com/title/tt0106977/?ref_=adv_li_tt',
 'https://www.imdb.com/title/tt0106918/?ref_=adv_li_tt',
 'https://www.imdb.com/title/tt0108160/?ref_=adv_li_tt',
 'https://www.imdb.com/title/tt0107211/?ref_=adv_li_tt',
 'https://www.imdb.com/title/tt0107206/?ref_=adv_li_tt',
 'https://www.imdb.com/title/tt0107798/?ref_=adv_li_tt',
 'https://www.imdb.com/title/tt0108052/?ref_=adv_li_tt',
 'https://www.imdb.com/title/tt0106582/?ref_=adv_li_tt',
 'https://www.imdb.com/title/tt0106965/?ref_=adv_li_tt',
 'https://www.imdb.com/title/tt0107818/?ref_=adv_li_tt',
 'https://www.imdb.com/title/tt0107688/?ref_=adv_li_tt',
 'https://www.imdb.com/title/tt0107048/?ref_=adv_li_tt',
 'https://www.imdb.com/title/tt0107050/?ref_=adv_li_tt',
 'https://www.imdb.com/title/tt0106611/?ref_=adv_li_tt',
 'https://www.imdb.com/title/tt0106673/?ref_=adv_li_tt',
 'https://www.imdb.com/title/tt

## Movie info Retrieval

In [7]:
# get the movie page inside the link

def get_movie(links):
    result = requests.get(links)
    soup = BeautifulSoup(result.text,'lxml')
    return soup

In [8]:
# get the name of the movie

def get_title(links):
    soupall = get_movie(links)
    title = soupall.find('div',class_="title_wrapper").h1.get_text().split('\xa0')[0]
    return title

In [9]:
get_title(Link_List_R[0])

'Jurassic Park'

In [10]:
# get the run time of the movie

def run_time(links):
    soupall = get_movie(links)
    time = soupall.find('time')
    
    if time is not None:
        time_r=time.get_text().replace(' ','').strip('\n')
        try:    
            int_time = int(re.split('\D',time_r)[0])*60+int(re.split('\D',time_r)[1])
        except:
            int_time = int(re.split('\D',time_r)[0])*60
    else:
        int_time = None
    return int_time

In [11]:
def release_date(links):
    soupall = get_movie(links)
    date = soupall.find(title="See more release dates")
    if date is not None:
        
        date = date.get_text().strip('\n').split()[0:3][::-1]
        date_str = '-'.join(date)
    else:
        date_str = None
    return date_str

In [12]:
release_date(Link_List_R[6426])

'2007-August-8'

In [13]:
# get the category of the movie

def categories(links):
    soupall = get_movie(links)
    cate = soupall.find('div', class_='subtext')
    category_list=[]
    
    for cat in cate.find_all('a'):
        if cat==[]:
            category= None
            category_list.append(category)
        else:
            category=cat.get_text(strip=True)
            category_list.append(category)

    if category_list==[]:
        category_list = category_list
    else:
        category_list.pop()
    return category_list

In [14]:
l=[]
for i in Link_List_R[6424:6430]:
    result = categories(i)
    l.append(result)
l

[['Comedy'],
 ['Comedy', 'Sport'],
 ['Comedy', 'Family'],
 ['Action', 'Comedy', 'Crime'],
 ['Adventure', 'Drama'],
 ['Comedy', 'Drama', 'Romance']]

In [15]:
categories(Link_List_R[6427])

['Action', 'Comedy', 'Crime']

In [16]:
#Get the country
def countries(links):
    soupall = get_movie(links)
    
    href_re=re.compile(pattern='\/search\/title\?country[\D]+dt_dt')

    country = soupall.find_all('a',attrs={"href":href_re})

## Extract the text
    name = [var.get_text(strip=True) for var in country]
    return name


In [17]:
countries(Link_List_R[5971])

['USA']

In [18]:
# get the languages of the movie

def languages(links):
    soupall = get_movie(links)
    
    href_re=re.compile(pattern='\/search\/title\?[\d\D]+language[\D]+dt_dt')

    language = soupall.find_all('a',attrs={"href":href_re})

## Extract the text
    name = [var.get_text(strip=True) for var in language]
    return name

In [19]:
Link_List_R[6427]

'https://www.imdb.com/title/tt0465602/?ref_=adv_li_tt'

In [20]:
languages(Link_List_R[6427])

['English', 'Italian']

In [21]:
def rated_class(links):
    soupall = get_movie(links)
    try:
        rated = soupall.find('div', class_='subtext').get_text().replace(' ','').replace('\n',' ').split()[0]
        rated = json.loads(soupall.find('script', {'type':"application/ld+json"}).get_text())['contentRating']
    except:
        rated = None
        
    return rated

In [22]:
rated_class(Link_List_R[2])

'PG-13'

In [23]:
rated_class(Link_List_R[6426])

'PG'

In [24]:
# get the ratings of the movie

def rating(links):
    soupall = get_movie(links)
    IMDB = soupall.find('span', itemprop="ratingValue")
    
    if IMDB is not None:
        IMDB = IMDB.get_text()
    else:
        IMDB = None
        
    Good_Meta = soupall.find('div', class_="metacriticScore score_favorable titleReviewBarSubItem")
    Soso_Meta = soupall.find('div', class_="metacriticScore score_mixed titleReviewBarSubItem")
    Bad_Meta = soupall.find('div', class_="metacriticScore score_unfavorable titleReviewBarSubItem")
    
    if Good_Meta is not None:
        Meta = Good_Meta.span.get_text()
    elif Soso_Meta is not None:
        Meta = Soso_Meta.span.get_text()
    elif Bad_Meta is not None:
        Meta = Bad_Meta.span.get_text()
    else:
        Meta = None
    
    
    return IMDB, Meta

In [25]:
Link_List_R[50]

'https://www.imdb.com/title/tt0106613/?ref_=adv_li_tt'

In [26]:
rating(Link_List_R[6427])

('6.7', '49')

In [27]:
rating(Link_List_R[50])

('3.9', None)

In [28]:
def staff(links):
    soupall = get_movie(links)
    
## Compile the regular expression matching '/name/nm0004874/?ref_=ttfc_fc'
    dir_re = re.compile(pattern="\/name\/nm[0-9]{7}/\?[\D]+ov_dr")
    wri_re = re.compile(pattern="\/name\/nm[0-9]{7}/\?[\D]+wr")
    act_re = re.compile(pattern='\/name\/nm[0-9]{7}/\?ref_=tt_cl_t\d+')
    
## Find the tag 'a' which have the particular form
    
    directors = soupall.find_all('a',attrs={"href":dir_re})
    writers = soupall.find_all('a',attrs={"href":wri_re})
    actors = soupall.find_all('a',attrs={"href":act_re})
    
## Extract the text
    director = [var.get_text(strip=True) for var in directors]
    writer = [var.get_text(strip=True) for var in writers]
    actor = [var.get_text(strip=True) for var in actors]
    return director, writer, actor

In [29]:
staff(Link_List_R[6427])

(['Michael Davis'],
 ['Michael Davis'],
 ['Clive Owen',
  'Paul Giamatti',
  'Monica Bellucci',
  'Stephen McHattie',
  'Greg Bryk',
  'Daniel Pilon',
  'Sidney Mende-Gibson',
  'Lucas Mende-Gibson',
  'Kaylyn Yellowlees',
  'Ramona Pringle',
  'Julian Richings',
  'Tony Munch',
  'Scott McCord',
  'Wiley M. Pickett',
  'Stephen R. Hart'])

In [30]:
# Get the opening weekend USA box offices
def get_budget(links):
    soupall = get_movie(links)
    B_O = soupall.find('h4', text='Budget:')
    if B_O is not None:
        Bud=B_O.parent.text
        Bud_1=Bud.replace('\n','').replace('\xa0','')
        Budget=re.split('[\s\:]',Bud_1)[1]
        return Budget
    else:
        return None

In [31]:
# Budget=re.search('\$[\d,]+', Bud).group().strip('$').replace(',','')

In [32]:
get_budget(Link_List_R[2])

'$44,000,000'

In [33]:
# Get the opening weekend USA box offices
def opening_USA(links):
    soupall = get_movie(links)
    tag1=soupall.find('h4', text='Opening Weekend USA:')
    if tag1 is not None:
        OpeningUSA=tag1.parent.text
        income=re.search('\$[\d,]+', OpeningUSA).group().strip(',')
    else:
        income = None
    return income

In [34]:
Link_List_R[2156]

'https://www.imdb.com/title/tt8924702/?ref_=adv_li_tt'

In [35]:
opening_USA(Link_List_R[2156])

In [36]:
opening_USA(Link_List_R[0])

'$50,159,460'

In [37]:
# Get the US box office
def USA(links):
    soupall = get_movie(links)
    tag2=soupall.find('h4', text='Gross USA:')
    if tag2 is not None:
        USBox=tag2.parent.text
        income=re.search('\$\[\d,]+', USBox)
        if income is not None:
            income = income.group()
        else:
            income = re.search(':[A-Za-z\W\s]+[\d,]+', USBox).group().replace(': ','')
        
    else:
        income = None
    return income

In [38]:
soupall = get_movie('https://www.imdb.com/title/tt0017860/?ref_=adv_li_tt')
tag2=soupall.find('h4', text='Gross USA:')
tag2

<h4 class="inline">Gross USA:</h4>

In [39]:
Link_List_R[8982]

'https://www.imdb.com/title/tt1748280/?ref_=adv_li_tt'

In [40]:
USA(Link_List_R[10])

'$77,698,625'

In [41]:
# Get the international box office
def international(links):
    soupall = get_movie(links)
    tag3=soupall.find('h4', text='Cumulative Worldwide Gross:')
    if tag3 is not None:
        inBox=tag3.parent.text
        income=re.search(':[A-Za-z\W\s]+[\d,]+',inBox).group().strip('\n').replace('\xa0','').replace(': ','')
        return income
    else:
        return None

In [42]:
Link_List_R[2296]

'https://www.imdb.com/title/tt0122690/?ref_=adv_li_tt'

In [43]:
international(Link_List_R[0])

'$1,029,153,882'

In [44]:
def comment(links):
    soupall = get_movie(links)
    
    # Vote Number Extraction
    votes = soupall.find(itemprop='ratingCount') 
    
    if votes is not None:
        votes = int(votes.get_text().replace(',',''))
    else:
        votes = None
    
    # Review & Critic Number Scraping
    r_c = soupall.find('div',class_='titleReviewbarItemBorder')             
    
    if r_c is not None:
        r_c_1 = r_c.find_all('a')
        name = [var.get_text(strip=True) for var in r_c_1]
    
    # Get the review number Extraction
        try:
            reviews = int((name[0].split())[0].replace(',','')) 
        except:
            reviews = None
    
    # Get the critic number Extraction
        try:
            critics = int((name[1].split())[0].replace(',','')) 
        except:
            critics = None
    else:
        reviews = None
        critics = None
    
    return votes, reviews, critics

In [45]:
Link_List_R[3513]

'https://www.imdb.com/title/tt0204933/?ref_=adv_li_tt'

In [46]:
comment(Link_List_R[5])

(56359, 124, 56)

## Function Combinations

In [47]:
# Get the information of the top 10k movies

def get_info(url):
    title_list=[]
    release_date_list=[]
    country_list=[]
    language_list=[]
    rated_list=[]
    runtime_list=[]
    category_list=[]
    IMDB_rating_list=[]
    Meta_rating_list=[]
    votes_list=[]
    reviews_list=[]
    critics_list=[]
    director_list=[]
    writer_list=[]
    actor_list=[]
    budget_list=[]
    opeing_USA_box_office_list=[]
    US_box_office_list=[]
    international_box_office_list=[]
    
#    for link in Link_List_R:
#        soupall = get_movie(link)
    sleep(6)
        
    for link in tqdm(url):
        soupall = get_movie(link)
        
    # Title
        title = get_title(link)
        title_list.append(title)
    # Release Date
        year = release_date(link)
        release_date_list.append(year)
    # Country
        country = countries(link)
        country_str = ', '.join(country)
        country_list.append(country_str)
    # Language
        language = languages(link)
        language_str = ', '.join(language)
        language_list.append(language_str)
    # Runtime
        runtime = run_time(link)
        runtime_list.append(runtime)
    # Category
        category = categories(link)
        category_str = ', '.join(category)
        category_list.append(category_str)
    # Rated Class
        rated = rated_class(link)
        rated_list.append(rated)
    # IMDB & Metacritic Rating
        IMDB, Metacritic = rating(link)
        IMDB_rating_list.append(IMDB)
        Meta_rating_list.append(Metacritic)
    # Num of Votes, Review & Critics
        votes, reviews, critics = comment(link)
        votes_list.append(votes)
        reviews_list.append(reviews)
        critics_list.append(critics)
    # Director, Writer & Actor
        director, writer, actor = staff(link)
        
        director_str = ', '.join(director)            # convert the list into multiple strings
        director_list.append(director_str)
    
        writer_str = ', '.join(writer)                # convert the list into multiple strings
        writer_list.append(writer_str)
    
        actor_str = ', '.join(actor)                  # convert the list into multiple strings
        actor_list.append(actor_str)
    # Budget
        budget = get_budget(link)
        budget_list.append(budget)
    # Opening USA Box Office
        opeing_USA_box_office = opening_USA(link)
        opeing_USA_box_office_list.append(opeing_USA_box_office)
    # US box office
        US_box_office = USA(link)
        US_box_office_list.append(US_box_office)
    # international box office
        international_box_office = international(link)
        international_box_office_list.append(international_box_office)
            
    return pd.DataFrame({'movie': title_list, 'release date': release_date_list,'country':country_list,\
                         'language':language_list, 'runtime': runtime_list, 'category': category_list,\
                         'rated': rated_list, 'IMDB rating':IMDB_rating_list, 'Metacritic rating': Meta_rating_list,\
                         'votes number': votes_list, 'reviews number': reviews_list, 'critics number': critics_list,\
                         'director':director_list, 'writer':writer_list, 'actor':actor_list,'budget':budget_list,\
                         'opeing USA box office':opeing_USA_box_office_list, 'US box office':US_box_office_list,\
                         'international box office':international_box_office_list})


### All the budget & international box office in the following df are string and needed to be converted

In [109]:
Year_1993 = get_info(Link_List_R[0:450])

In [124]:
Year_1993.iloc[200::]

,movie,release date,country,language,runtime,category,rated,IMDB rating,Metacritic rating,votes number,reviews number,critics number,director,writer,actor,budget,opeing USA box_office,US box office,international box office
200,The Ballad of Little Jo,1993-August-20,USA,English,121.0,"Drama, Romance, Western",R,6.8,None,1284.0,24.0,15.0,Maggie Greenwald,Maggie Greenwald,"Suzy Amis, Bo Hopkins, Ian McKellen, David Chu...",None,"$20,773","$543,091","$565,530"
201,Caro diario,1993-November-12,"Italy, France","Italian, English, Mandarin",100.0,"Biography, Comedy, Drama",Unrated,7.3,None,6364.0,25.0,27.0,Nanni Moretti,Nanni Moretti,"Nanni Moretti, Giovanna Bozzolo, Sebastiano Na...",None,None,"$472,598",None
202,American Cyborg: Steel Warrior,1994-January-7,USA,English,94.0,"Action, Sci-Fi",R,3.7,None,1147.0,30.0,21.0,Boaz Davidson,"Bill Crounse, Boaz Davidson","Joe Lara, Nicole Hansen, John Saint Ryan, Jose...",None,None,"$447,784",None
203,In a Moment of Passion,1993-March-19,"Poland, Germany, USA","English, Polish",100.0,"Drama, Thriller",R,3.6,None,36.0,3.0,NaN,Zbigniew Kaminski,"Charles Haigh, Zbigniew Kaminski","Chase Masterson, Vivian Schilling, Julie Arask...",None,None,"$436,132",None
204,Fiorile,1993-March-26,"Italy, France, Germany","Italian, French",119.0,Drama,PG-13,7.0,None,1007.0,10.0,10.0,"Paolo Taviani, Vittorio Taviani","Paolo Taviani, Vittorio Taviani","Claudio Bigagli, Galatea Ranzi, Michael Vartan...",None,None,"$429,184",None
205,Body Snatchers,1993-June-9,USA,English,87.0,"Horror, Sci-Fi",R,5.9,65,15214.0,87.0,68.0,Abel Ferrara,"Jack Finney, Raymond Cistheri","Terry Kinney, Meg Tilly, Gabrielle Anwar, Reil...","$13,000,000",None,"$428,868",None
206,Killing Zoe,1994-August-31,"France, USA","English, French",96.0,"Crime, Thriller",R,6.5,None,18757.0,98.0,37.0,Roger Avary,Roger Avary,"Eric Stoltz, Martin Raymond, Eric Pascal Chalt...","$1,500,000",None,"$418,953",None
207,Golden Gate,1994-January-28,USA,English,91.0,"Crime, Drama",R,5.3,None,669.0,9.0,5.0,John Madden,David Henry Hwang,"Matt Dillon, Joan Chen, Elizabeth Morehead, Br...",None,None,"$395,105",None
208,Germinal,1993-September-29,"France, Belgium, Italy",French,160.0,"Drama, Romance",R,7.1,None,4218.0,19.0,8.0,Claude Berri,"Claude Berri, Arlette Langmann","Miou-Miou, Renaud, Jean Carmet, Judith Henry, ...","FRF164,000,000",None,"$378,854",None
209,Lan feng zheng,1994-February-2,"China, Hong Kong",Mandarin,140.0,Drama,None,7.7,None,2371.0,18.0,15.0,Zhuangzhuang Tian,Xiao Mao,"Tian Yi, Wenyao Zhang, Xiaoman Chen, Liping Lü...",None,None,"$355,974",None


In [122]:
Year_1993[Year_1993['release date'].notnull()]

,movie,release date,country,language,runtime,category,rated,IMDB rating,Metacritic rating,votes number,reviews number,critics number,director,writer,actor,budget,opeing USA box_office,US box office,international box office
0,Jurassic Park,1993-June-11,USA,"English, Spanish",127.0,"Adventure, Sci-Fi, Thriller",PG-13,8.1,68,773402.0,1010.0,343.0,Steven Spielberg,"Michael Crichton, Michael Crichton","Sam Neill, Laura Dern, Jeff Goldblum, Richard ...","$63,000,000","$50,159,460","$402,453,882","$1,029,153,882"
1,Mrs. Doubtfire,1993-November-24,USA,English,125.0,"Comedy, Drama, Family",PG-13,7.0,53,217547.0,252.0,55.0,Chris Columbus,"Anne Fine, Randi Mayem Singer","Robin Williams, Sally Field, Pierce Brosnan, H...","$25,000,000",None,"$219,200,000","$441,286,195"
2,The Fugitive,1993-August-6,USA,"English, Polish, Spanish",130.0,"Action, Crime, Drama",PG-13,7.8,87,243089.0,295.0,123.0,Andrew Davis,"Jeb Stuart, David Twohy","Harrison Ford, Tommy Lee Jones, Sela Ward, Jul...","$44,000,000","$23,758,855","$183,875,760","$368,875,760"
3,The Firm,1993-June-30,USA,English,154.0,"Drama, Mystery, Thriller",R,6.8,58,104963.0,151.0,65.0,Sydney Pollack,"John Grisham, David Rabe","Tom Cruise, Jeanne Tripplehorn, Gene Hackman, ...","$42,000,000","$25,400,000","$158,348,367","$270,248,367"
4,Sleepless in Seattle,1993-June-25,USA,English,105.0,"Comedy, Drama, Romance",PG,6.8,72,143571.0,233.0,77.0,Nora Ephron,"Jeff Arch, Nora Ephron","Tom Hanks, Ross Malinger, Rita Wilson, Victor ...","$21,000,000",None,"$126,533,006","$101,200,000"
5,Indecent Proposal,1993-April-7,USA,English,117.0,"Drama, Romance",R,5.9,35,56359.0,124.0,56.0,Adrian Lyne,"Jack Engelhard, Amy Holden Jones","Robert Redford, Demi Moore, Woody Harrelson, S...","$38,000,000",None,"$106,614,100","$266,600,000"
6,In the Line of Fire,1993-July-9,USA,English,128.0,"Action, Crime, Drama",R,7.2,74,84376.0,146.0,73.0,Wolfgang Petersen,Jeff Maguire,"Clint Eastwood, John Malkovich, Rene Russo, Dy...","$40,000,000",None,"$102,314,283","$85,000,000"
7,The Pelican Brief,1993-December-17,USA,English,141.0,"Crime, Drama, Mystery",PG-13,6.6,50,69321.0,101.0,49.0,Alan J. Pakula,"John Grisham, Alan J. Pakula","Julia Roberts, Denzel Washington, Sam Shepard,...","$45,000,000","$16,864,404","$100,768,056","$195,268,056"
8,Schindler's List,1993-December-25,USA,"English, Hebrew, German, Polish",195.0,"Biography, Drama, History",R,8.9,93,1071427.0,1553.0,217.0,Steven Spielberg,"Thomas Keneally, Steven Zaillian","Liam Neeson, Ben Kingsley, Ralph Fiennes, Caro...","$22,000,000","$656,636","$96,067,179","$221,000,000"
9,Cliffhanger,1993-May-28,"USA, Italy, France",English,113.0,"Action, Adventure, Thriller",R,6.4,60,110389.0,181.0,86.0,Renny Harlin,"John Long, Michael France","Sylvester Stallone, John Lithgow, Michael Rook...","$70,000,000","$20,458,022","$84,049,211","$171,000,000"


In [48]:
Year_1994 = get_info(Link_List_R[450:900])

In [49]:
Year_1994.to_csv('Year_1994.csv',index=False)

In [51]:
Year_1995_to_2000 = get_info(Link_List_R[900:3600])

In [52]:
Year_1995_to_2000.to_csv('Year_1995_to_2000.csv',index=False)

In [53]:
Year_2001_to_2003 = get_info(Link_List_R[3600:4950])

In [54]:
Year_2001_to_2003.to_csv('Year_2001_to_2003.csv',index=False)

In [55]:
Year_2004_to_2009 = get_info(Link_List_R[4950:7650])

In [57]:
Year_2004_to_2009.to_csv('Year_2004_to_2009.csv',index=False)

In [68]:
Link_List_R[11695:11701]

['https://www.imdb.com/title/tt9865284/?ref_=adv_li_tt',
 'https://www.imdb.com/title/tt9862466/?ref_=adv_li_tt',
 'https://www.imdb.com/title/tt9858794/?ref_=adv_li_tt',
 'https://www.imdb.com/title/tt9857770/?ref_=adv_li_tt',
 'https://www.imdb.com/title/tt9856680/?ref_=adv_li_tt']

In [69]:
Year_2010_to_2019 = get_info(Link_List_R[7650:11700])

In [70]:
Year_2010_to_2019.to_csv('Year_2010_to_2019.csv',index=False)

In [154]:
table = pd.read_csv('Top_1.csv')
table['release date']=pd.to_datetime(table['release date'],format='%Y-%B-%d')
table.head()
table.to_csv('Top_1.csv',index=False)

,movie,release date,country,language,runtime,category,rated,IMDB rating,Metacritic rating,votes number,reviews number,critics number,director,writer,actor,budget,opeing_USA_box_office,US_box_office,international_box_office
0,Star Wars: Episode VII - The Force Awakens,2015-12-18,USA,English,136,"Action, Adventure, Fantasy",PG-13,8.0,81.0,775332,4448,869,J.J. Abrams,"Lawrence Kasdan, J.J. Abrams","Harrison Ford, Mark Hamill, Carrie Fisher, Ada...","$245,000,000",247966675.0,936662225.0,"$2,068,223,624"
1,Avatar,2009-12-18,"UK, USA","English, Spanish",162,"Action, Adventure, Fantasy",PG-13,7.8,83.0,1031518,3217,727,James Cameron,James Cameron,"Sam Worthington, Zoe Saldana, Sigourney Weaver...","$237,000,000",77025481.0,760507625.0,"$2,787,965,087,"
2,Black Panther,2018-02-16,USA,"English, Swahili, Nama, Xhosa, Korean",134,"Action, Adventure, Sci-Fi",PG-13,7.3,88.0,481847,2854,637,Ryan Coogler,"Ryan Coogler, Joe Robert Cole","Chadwick Boseman, Michael B. Jordan, Lupita Ny...","$200,000,000",202003951.0,700059566.0,"$1,347,071,259"
3,Avengers: Infinity War,2018-04-27,USA,English,149,"Action, Adventure, Sci-Fi",PG-13,8.5,68.0,605973,3830,560,"Anthony Russo, Joe Russo","Christopher Markus, Stephen McFeely","Robert Downey Jr., Chris Hemsworth, Mark Ruffa...","$321,000,000",257698183.0,678815482.0,"$2,048,709,917"
4,Titanic,1997-12-19,USA,"English, Swedish",194,"Drama, Romance",PG-13,7.8,75.0,942899,2657,318,James Cameron,James Cameron,"Leonardo DiCaprio, Kate Winslet, Billy Zane, K...","$200,000,000",28638131.0,659325379.0,"$2,186,772,302,"


In [151]:
table[table['budget'].str.contains('\$\d+') == False]

,movie,release date,country,language,runtime,category,rated,IMDB rating,Metacritic rating,votes number,reviews number,critics number,director,writer,actor,budget,opeing_USA_box_office,US_box_office,international_box_office
80,Harry Potter and the Deathly Hallows: Part 1,2010-11-19,"UK, USA",English,146,"Adventure, Fantasy, Mystery",PG-13,7.7,65.0,418470,651,435,David Yates,"Steve Kloves, J.K. Rowling","Bill Nighy, Emma Watson, Richard Griffiths, Ha...","£150,000,000",125017327.0,295983305.0,"$960,283,305"


In [62]:
Top_2 = get_info(Link_List_R[500:2000])

In [64]:
Top_2.to_csv('Top_2.csv',index=False)

In [44]:
Top_3 = get_info(Link_List_R[2000:3000])

In [45]:
Top_3.to_csv('Top_3.csv',index=False)

In [48]:
Link_List_R[3513]

'https://www.imdb.com/title/tt6131632/?ref_=adv_li_tt'

In [58]:
Top_4 = get_info(Link_List_R[3000:3500])

In [59]:
Top_4.to_csv('Top_4.csv',index=False)

In [65]:
Top_5 = get_info(Link_List_R[3500:5000]) 

In [66]:
Top_5.to_csv('Top_5.csv',index=False)

In [84]:
Link_List_R[5803]

'https://www.imdb.com/title/tt0461286/?ref_=adv_li_tt'

In [93]:
Top_6 = get_info(Link_List_R[5000:6000])

In [95]:
Top_6.to_csv('Top_6.csv',index=False)

In [97]:
Link_List_R[6427]

'https://www.imdb.com/title/tt0102924/?ref_=adv_li_tt'

In [208]:
Top_7 = get_info(Link_List_R[6000:7000])

In [209]:
Top_7.to_csv('Top_7.csv',index=False)

In [212]:
Link_List_R[8982]

'https://www.imdb.com/title/tt0017860/?ref_=adv_li_tt'

In [230]:
Top_8 = get_info(Link_List_R[7000:8000])

In [231]:
Top_8.to_csv('Top_8.csv',index=False)

In [232]:
Top_9 = get_info(Link_List_R[8000:9000])

In [233]:
Top_9.to_csv('Top_9.csv',index=False)

In [234]:
Top_10 = get_info(Link_List_R[9000:10000])

In [235]:
Top_10.to_csv('Top_10.csv',index=False)

## Retrieve Staff List

### Top Directors

In [139]:
def dirinfo(page):
    url = 'https://www.boxofficemojo.com/people/?view=Director&pagenum={}&sort=sumgross&order=DESC&&p=.htm'.format(page)
    result=requests.get(url)
    soup=BeautifulSoup(result.text,'lxml')
    
    return soup

def information(page):
    soup = dirinfo(page)
    l=[]    
    for i in soup.find_all('font', size="2"):
        n=i.get_text()
        l.append(n)
    return l      

In [140]:
director_list=[]
movie_list=[]
average_list=[]
gross_list=[]
for page in range(1,6):
    ls=information(page)
    director = ls[9::7]
    del director[-1]
    num_movies=ls[11::7]
    average=ls[12::7]
    gross=ls[10::7]
    director_list+= director
    movie_list+= num_movies
    average_list+=average
    gross_list += gross
    

dir_df = pd.DataFrame({'director':director_list,'# of movies':movie_list,'average earned':average_list,'Gross':gross_list})\
         .drop_duplicates().reset_index().drop(columns='index')

### Top Actors & Actresses List

In [141]:
def actorinfo(page):
    url = 'https://www.boxofficemojo.com/people/?view=Actor&pagenum={}&sort=sumgross&order=DESC&&p=.htm'.format(page)
    result=requests.get(url)
    soup=BeautifulSoup(result.text,'lxml')
    
    return soup

def information(page):
    soup = actorinfo(page)
    l=[]    
    for i in soup.find_all('font', size="2"):
        n=i.get_text()
        l.append(n)
    return l      

In [142]:
actor_list=[]
movie_list=[]
average_list=[]
gross_list=[]
for page in range(1,6):
    ls=information(page)
    actor = ls[9::7]
    del actor[-1]
    num_movies=ls[11::7]
    average=ls[12::7]
    gross=ls[10::7]
    actor_list+= actor
    movie_list+= num_movies
    average_list+=average
    gross_list += gross
    

act_df = pd.DataFrame({'actor':actor_list,'# of movies':movie_list,'average earned':average_list,'Gross':gross_list})\
         .drop_duplicates().reset_index().drop(columns='index')

In [143]:
act_df.tail()

,actor,# of movies,average earned,Gross
445,Josh Lucas,24,$33.0,$791.1
446,Uma Thurman,34,$23.2,$789.1
447,Katherine Heigl,18,$43.6,$785.4
448,Elizabeth Debicki,8,$97.8,$782.3
449,Chris O'Donnell,19,$41.0,$779.3


In [144]:
act_df['actor'].nunique()

450

### Top Writer List

In [145]:
def writerinfo(page):
    url = 'https://www.boxofficemojo.com/people/?view=Writer&pagenum={}&sort=sumgross&order=DESC&&p=.htm'.format(page)
    result=requests.get(url)
    soup=BeautifulSoup(result.text,'lxml')
    
    return soup

def information(page):
    soup = writerinfo(page)
    l=[]    
    for i in soup.find_all('font', size="2"):
        n=i.get_text()
        l.append(n)
    return l      

In [146]:
writer_list=[]
movie_list=[]
average_list=[]
gross_list=[]
for page in range(1,6):
    ls=information(page)
    writer = ls[9::7]
    del writer[-1]
    num_movies=ls[11::7]
    average=ls[12::7]
    gross=ls[10::7]
    writer_list+= writer
    movie_list+= num_movies
    average_list+=average
    gross_list += gross
    

wri_df = pd.DataFrame({'writer':writer_list,'# of movies':movie_list,'average earned':average_list,'Gross':gross_list})\
         .drop_duplicates().reset_index().drop(columns='index')

In [147]:
wri_df.tail()

,writer,# of movies,average earned,Gross
445,Bryan Singer,1,$146.4,$146.4
446,Matt Damon,3,$48.8,$146.3
447,Richard D'Ovidio,3,$48.5,$145.5
448,Rodo Sayagues Mendez,2,$71.7,$143.5
449,John-Henry Butterworth,3,$46.8,$140.5


In [148]:
act_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450 entries, 0 to 449
Data columns (total 4 columns):
actor             450 non-null object
# of movies       450 non-null object
average earned    450 non-null object
Gross             450 non-null object
dtypes: object(4)
memory usage: 14.1+ KB


In [149]:
def dollarprocess(df):
    for column in ['Gross', 'average earned']:
        df[column]=df[column].str.replace(',','').str.strip('$')
        df[column]=df[column].astype(float)
    return df.head()

dollarprocess(dir_df)
dollarprocess(wri_df)
dollarprocess(act_df)

,actor,# of movies,average earned,Gross
0,Samuel L. Jackson,76,82.0,6228.6
1,Harrison Ford,42,118.2,4963.8
2,"Robert Downey, Jr.",55,90.2,4960.4
3,Tom Hanks,48,96.0,4606.8
4,Morgan Freeman,64,71.5,4577.4


In [150]:
def number(df):
    for column in ['# of movies']:
        df[column]=df[column].astype(int)

number(wri_df)
number(dir_df)
number(act_df)

In [154]:
dir_df=dir_df[dir_df['Gross']>500]
wri_df=wri_df[wri_df['Gross']>500]
act_df=act_df[act_df['Gross']>1000]

In [155]:
dir_df.to_csv('director income.csv',index=False)
wri_df.to_csv('writer income.csv',index=False)
act_df.to_csv('actor income.csv',index=False)